## Calculate GGT

Take the csv(s) downloaded from the Sharesies app, each detailing transactions for a given year.

Group trades by companies. Calculate the profit/loss on trades. Calculate the CGT, depending on how long the shares were held for.

In [ ]:
import pathlib
import pandas
import os

# https://stackoverflow.com/questions/55491046/how-to-set-the-running-file-path-of-jupyter-in-vscode 
# set VS Code "Notebook File Root" to ${workspaceFolder}
from shares.config import SHARESIES as config

# enable dict.notation
import dotsi
config = dotsi.Dict(config)

### User Configuration

User-controllable configuration is stored in variables with CAPTIAL LETTERS.

In [ ]:
DATA_DIR = pathlib.Path(os.path.expanduser("~/data/"))
FILE = 'transaction-report.csv'

AUD_SHARE_TOTAL = 'AUD Amount'
AUD_FEE = 'AUD Transaction Fee'

In [ ]:
df = pandas.read_csv(DATA_DIR.joinpath(FILE))
# shows a sample of the data
df.head(5)

### Calculating code

In [ ]:
def add_aud_translation(df):
    """
    Add columms for AUD, for all base currency.

    In the current export I have from sharesies, the SELL order in USD does not contain an exchange rate.
    """
    df[AUD_SHARE_TOTAL] = 0.0
    df[AUD_FEE] = 0.0
    
    for ridx, row in df.iterrows():

        if row[config.CURRENCY] != 'aud':
            if row[config.BUYSELL] == 'BUY':
                # this works for buy rows, but the sell translation is not captured
                df.loc[ridx, AUD_SHARE_TOTAL] = row[config.SHARE_PRICE_TOTAL] / row[config.EXCHANGE_RATE]
                # assuming the fee is in the currency recorded
                df.loc[ridx, AUD_FEE] = row[config.FEE] / row[config.EXCHANGE_RATE]
            else:
                # REQUIRES GETTING CURRENCY CONVERSION
                df.loc[ridx, AUD_SHARE_TOTAL] = row[config.SHARE_PRICE_TOTAL] / row[config.EXCHANGE_RATE]
                df.loc[ridx, AUD_FEE] = row[config.FEE] / row[config.EXCHANGE_RATE]
        else:
            df.loc[ridx, AUD_SHARE_TOTAL] = row[config.SHARE_PRICE_TOTAL]
            df.loc[ridx, AUD_FEE] = row[config.FEE]

In [ ]:
def gather(df, code, buy_or_sell='BUY', display_df=True):
    """
    Collate data for a given share code, and given transaction type.

    Returns a view of the data for the given code and BUY or SELL.
    """
    c_df = df.loc[df[config.TICKER] == code]
    c_df_trade = c_df.loc[c_df['Transaction type'].str.contains(buy_or_sell)].sort_values(config.TRADE_DATE)
    if display_df:
        display(c_df_trade)
    return c_df_trade

In [ ]:
def calculate(df_buy, df_sell, display_df=True):
    """
    Calculate the overall profit (or loss) for a given stock, in order to feed into the CGT calculation.

    The buying cost is the cost of the shares, plus any transaction fees for buying.
    The sale amount is the amount received for the sale of the shares, minus any costs for selling.

    Profit/loss = sale amount - sale fees - (buy amount + buy fees)

    If converting to aud, only the total profit/loss realised should be converted

    CGT will apply on any overall profit. Overall capital losses can be carried forward for future years.
    The CGT discount applies for shares held greater than 12 months - TODO.
    """
    missing = 'MISSING BUY OR SELL'
    exchange = 'EXCHANGE FIX REQUIRED'
    ok = 'OK'

    status = ok

    ticker = df_sell[config.TICKER].unique()

    buy_n = df_buy[config.QUANTITY].sum(min_count=1)
    sell_n = df_sell[config.QUANTITY].sum(min_count=1)

    buy_currency = df_buy[config.CURRENCY].unique()
    sell_currency = df_sell[config.CURRENCY].unique()

    def check_currency(buy_currency, sell_currency):
        if len(buy_currency) > 1 or len(sell_currency) > 1 or len(buy_currency) != len(sell_currency) or buy_currency != sell_currency:
            print("WARNING: Mixed currencies in buy or sell.")
            return False
        return True

    if pandas.isnull(buy_n):
        print(f"This ticker has no purchase records: {ticker[0]}")
        status = missing
    else:
        if pandas.isnull(sell_n):
            print(f"This ticker has no sale records: {df_buy['Instrument code'].unique()[0]}")
            status = missing
        else:
            if not check_currency(buy_currency, sell_currency):
                status = exchange


    buy_cost = df_buy[AUD_SHARE_TOTAL].sum(min_count=1)
    buy_cost_fees = df_buy[AUD_FEE].sum(min_count=1)
    sell_return = df_sell[AUD_SHARE_TOTAL].sum(min_count=1)
    sell_fees = df_sell[AUD_FEE].sum(min_count=1)

    buy_cost_total = buy_cost + buy_cost_fees
    sell_return_total = sell_return - sell_fees
    if pandas.isnull(sell_return_total):
        status = exchange


    remaining_shares = round(buy_n - sell_n, 6)

    df_summary = pandas.DataFrame(
        {
            'Buy trades': df_buy.shape[0],
            'Sell trades': df_sell.shape[0],
            'Buy shares': buy_n,
            'Sell shares': sell_n,
            'Remaining shares': remaining_shares,
            'Buy cost total': buy_cost_total,
            'Sell return total': sell_return_total,
            'Profit (loss)': sell_return_total - buy_cost_total,
            'Currency': sell_currency,
            'Ticker': ticker,
            'Status': status
        }
    )

    if remaining_shares != 0:
        print("WARNING: Shares remain, so not useful for CGT")

    if display_df:
        display(df_summary)

    return df_summary

### Perform the calculations

In [ ]:
df_calc = None

add_aud_translation(df)

for code in df['Instrument code'].sort_values().unique():
    df_buy = gather(df, code, 'BUY', display_df=False)
    df_sell = gather(df, code, 'SELL', display_df=False)

    # list only tickers that had SELL transactions
    if df_sell.shape[0] > 0:
        print(f"\nProcessing {code} which has had SELL transactions...")
        df_all = gather(df,code, '', display_df=False)

        df_calculate = calculate(df_buy, df_sell)
        if df_calc is None:
            df_calc = df_calculate
        else:
            df_calc = pandas.concat([df_calc, df_calculate])

In [ ]:
df_calc.loc[df_calc['Status'] == 'OK']

In [ ]:
df_calc.loc[df_calc['Status'] != 'OK']